In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import requests
import plotly.express as px
from time import sleep

In [6]:
api_key = "ioCjwA1EbBvFdjMSq9JkwLYee9E8IXeh"        #romanbanenko1

api_key = "bW80BB19SE9iIq1Bt4bsISq6OjTIN9Yw"        #mishashegoleh

#https://site.financialmodelingprep.com/developer/docs/dashboard

# Grouping

In [42]:
sectors_forecast = {
    "bigtech": "^SP500-45",
    "staples": "^SP500-30",
    "Utilities": "^SP500-55",
    "finance": "^GSPC",
    "health": "^SP500-35",
    "discretionary": "^SP500-25",
    "real estate": "^SP500-60",
}


sectors_names = {
    "^SP500-45": "bigtech",
    "^SP500-30": "staples",
    "^SP500-55": "Utilities",
    "^GSPC": "finance",
    "^SP500-35": "health",
    "^SP500-25": "discretionary",
    "^SP500-60": "real estate"
}


sectors_annual_average_return = {           #BASED ON HISTORY DATA
    "^SP500-45": 0.275590,
    "^SP500-30": 0.086390,
    "^SP500-55": 0.060968,
    "^GSPC": 0.165529,
    "^SP500-35": 0.114558,
    "^SP500-25": 0.215885,
    "^SP500-60": 0.074752,
}

inflation_rate = 0.04

t = 5


sectors_trailingPE = {           #ENTER MANUALLY
    "^SP500-45": 28.5,
    "^SP500-30": 21.8,
    "^SP500-55": 18.9,
    "^GSPC": 17.7,
    "^SP500-35": 18.0,
    "^SP500-25": 27.8,
    "^SP500-60": 39.7,
}

In [27]:
group_by_metrics = {
    "Market Price 5Y Change": "G",
    "EPS 5Y Change": "G",
    "YoY Growth": "G",
    "Multiplicative method": "G",
    "CAGR method": "G",
    
    "P/S": "V",
    "P/E": "V",
    "EV/EBITDA": "V",
    "PEG": "V",
    "ROI": "V",
    "ROA": "V",
    "ROE": "V",
    
    # "Discounted Model": "V",  # <-- использовано для прогноза по денежным потокам
    
    "DPR": "Q",
    "Cash Flow 5Y Change": "Q",
    "EBITDA 5Y Change": "Q",
    "Sales 5Y Change": "Q",
    "Net Income 5Y Change": "Q",
    "Net Debt/EBITDA": "Q",
    "Free Cash Flow": "Q",  # <-- дополнительная метрика
    "Dividend Yield": "Q"  # <-- дополнительная метрика
}

group_by_aspects = {
    "Market Price 5Y Change": "Historical Growth",
    "EPS 5Y Change": "Historical Growth",
    "YoY Growth": "Historical Growth",
    
    "Multiplicative method": "Future Growth Predictions",
    "CAGR method": "Future Growth Predictions",
    
    "P/S": "Market-Based Ratios",
    "P/E": "Market-Based Ratios",
    "EV/EBITDA": "Market-Based Ratios",
    "PEG": "Market-Based Ratios",
    
    # "ROI": "Return-Based Ratios",
    "ROA": "Return-Based Ratios",
    "ROE": "Return-Based Ratios",
    
    # "Discounted Model": "V",  # <-- использовано для прогноза по денежным потокам
    
    "DPR": "Dividend Metrics",
    "Dividend Yield": "Dividend Metrics",  # <-- добавлено для дивидендов

    "Cash Flow 5Y Change": "Profitability & Stability",
    # "Net Income 5Y Change": "Profitability & Stability",
    "EBITDA 5Y Change": "Profitability & Stability",
    
    "Sales 5Y Change": "Size & Growth Stability",
    
    "Net Debt/EBITDA": "Leverage",
    "FCF Yield": "Profitability & Stability"  # <-- добавлено для фри-кэш-флоу
}


aspects_metrics = {
    "Historical Growth": ["Market Price 5Y Change", "EPS 5Y Change", "YoY Growth"],
    "Future Growth Predictions": ["Multiplicative method", "CAGR method"],
    "Market-Based Ratios": ["P/S", "P/E", "EV/EBITDA", "PEG"],
    # "Return-Based Ratios": ["ROI", "ROA", "ROE"],
    "Return-Based Ratios": ["ROA", "ROE"],
    # "Valuation": ["Discounted Model"],
    "Dividend Metrics": ["DPR", "Dividend Yield"],
    # "Profitability & Stability": ["Cash Flow 5Y Change", "Net Income 5Y Change", "EBITDA 5Y Change", "FCF Yield"],
    "Profitability & Stability": ["Cash Flow 5Y Change", "EBITDA 5Y Change", "FCF Yield"],
    "Size & Growth Stability": ["Sales 5Y Change"],
    "Leverage": ["Net Debt/EBITDA"]
}


gqv_for_aspects = {
    "G": ["Historical Growth", "Future Growth Predictions"],
    # "V": ["Market-Based Ratios", "Valuation"],
    "V": ["Market-Based Ratios"],
    "Q": ["Dividend Metrics", "Profitability & Stability", "Size & Growth Stability", "Leverage"]
}

# PARTIALLY

## Forecasting price (Multiplicative method, CAGR method)

In [9]:
def multiplicative_method(stock_growth, industry_growth, beta, dividend_yield=0, t=1, inflation_rate=0):

    # Вычисляем факторы роста
    aim_growth_upper = ((1 + stock_growth) * ((1 + 0.01 * beta) * (1 + industry_growth)))
    aim_growth_lower = ((1 + stock_growth) * ((1 - 0.01 * beta) * (1 + industry_growth)))

    # Учитываем дивиденды, если применимо
    if dividend_yield > 0:
        inflation_adjustment = (1 - inflation_rate) ** t
        dividend_factor = dividend_yield * t * inflation_adjustment
        aim_growth_upper *= 1 + dividend_factor
        aim_growth_lower *= 1 + dividend_factor


    # Средняя цена
    avg_growth = (aim_growth_upper + aim_growth_lower) / 2

    return avg_growth, aim_growth_lower, aim_growth_upper



def cagr_revenue_model(revenue_current, cagr_last_5y, ev_current, beta, t=5):
    # Прогнозируем доход через t лет
    revenue_forecast = revenue_current * (1 + cagr_last_5y) ** t


    # Прогнозируем EV через t лет
    ev_forecast = revenue_forecast * (ev_current / revenue_current)

    # Прогнозируем CAGR через t лет
    cagr_forecast = (ev_forecast / ev_current) ** (1 / t) - 1

    # Вычисляем факторы роста через t лет
    aim_growth_upper = ((1 + cagr_forecast) ** t - 1) * (1 + 0.01 * beta)
    aim_growth_lower = ((1 + cagr_forecast) ** t - 1) * (1 - 0.01 * beta)

    # Средняя цена за t лет
    avg_growth = (aim_growth_upper + aim_growth_lower) / 2
    return avg_growth, aim_growth_lower, aim_growth_upper

## GET DATA

In [ ]:
stock = yf.Ticker('SEIC')
info = stock.info
print(info.get("totalDebt")/501250000)
print(929196998656/78055000000)
# stock.history(period="5y")["Close"].pct_change().sum()


0.06498354114713217
11.904387914368074


In [10]:
multiplicative_forecasting = {}     #ticker: [average, upper, lower]
cagr_forecasting = {}     #ticker: [average, upper, lower]

def get_data(sector_ticker, tickers):

    data = []

    for ticker in tickers:

        print(ticker)

        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Получаем финансовые данные за последние 5 лет из FMP
        url_financials = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=5&apikey={api_key}"
        url_cash_flow = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?limit=5&apikey={api_key}"
        url_profile = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api_key}"
        
        financials = requests.get(url_financials).json()
        print(financials)
        cash_flows = requests.get(url_cash_flow).json()
        profile = requests.get(url_profile).json()

        # Проверка на наличие данных
        ebitda_5y_change = None
        cash_flow_5y_change = None
        sales_5y_change = None
        net_income_5y_change = None
        market_cap_5y_change = None

        beta = info.get("beta")
        dividend_yield = info.get("dividendYield", 0)
        ev_current = float(info.get("enterpriseValue", 1))
        revenue_current = float(info.get("totalRevenue", 1))
        pe_company = info.get("trailingPE", 0)

        #Price Forecasting
        revenue_5y_ago = financials[4]["revenue"]
        # Проверка на наличие выручки за 5 лет назад
        if revenue_5y_ago > 0:
            cagr_last_5y = ((revenue_current / revenue_5y_ago) ** (1 / t)) - 1
        else:
            cagr_last_5y = 0


        sector_pe = sectors_trailingPE[sector_ticker]  # Можно интегрировать API для получения среднего значения P/E сектора
        # Проверка на существование данных
        if sector_pe is not None and sector_pe > 0:
            # Расчет разницы P/E
            pe_diff_percentage = ((pe_company - sector_pe) / sector_pe)
        else:
            pe_diff_percentage = None  # Если данных нет, оставляем None

        

        multiplicative_forecasting[ticker] = list(multiplicative_method(pe_diff_percentage, sectors_annual_average_return[sector_ticker], beta, dividend_yield, t, inflation_rate))
        cagr_forecasting[ticker] = list(cagr_revenue_model(revenue_current, cagr_last_5y, ev_current, beta, t))
        


        if len(financials) >= 2:

            ebitda_current = float(financials[0].get("ebitda", 0))
            ebitda_5y_ago = float(financials[-1].get("ebitda", 1))

            if (ebitda_5y_ago == 0):
                print('hgggggggg')
                ebitda_current = float(financials[0].get("ebitda", 0))
                ebitda_5y_ago = float(financials[-3].get("ebitda", 1))


            # EBITDA 5Y Change
            
            print(ebitda_5y_ago)
            ebitda_5y_change = ((ebitda_current - ebitda_5y_ago) / ebitda_5y_ago)

            # Sales 5Y Change
            sales_current = float(financials[0].get("revenue", 0))
            sales_5y_ago = float(financials[-1].get("revenue", 1))
            sales_5y_change = ((sales_current - sales_5y_ago) / sales_5y_ago)

            # Net Income 5Y Change
            net_income_current = float(financials[0].get("netIncome", 0))
            net_income_5y_ago = float(financials[-1].get("netIncome", 1))
            net_income_5y_change = ((net_income_current - net_income_5y_ago) / net_income_5y_ago)

        if len(cash_flows) >= 2:
            # Cash Flow 5Y Change
            cash_flow_current = float(cash_flows[0].get("operatingCashFlow", 0))
            cash_flow_5y_ago = float(cash_flows[-1].get("operatingCashFlow", 1))
            cash_flow_5y_change = ((cash_flow_current - cash_flow_5y_ago) / cash_flow_5y_ago) 

            #Free CashFlow
            free_cash_flow = cash_flows[0].get("freeCashFlow", 0)

        if profile:
            # Market Cap 5Y Change

            market_cap_current = profile[0].get("mktCap")
            market_cap_5y_ago = profile[0].get("mktCap5Y")
            if market_cap_current and market_cap_5y_ago:
                market_cap_5y_change = ((market_cap_current - market_cap_5y_ago) / market_cap_5y_ago) 


        FCF_yield = (free_cash_flow / market_cap_current)


        ev_to_ebitda = info.get("enterpriseToEbitda")
        pe =  info.get("trailingPE")
        peg = info.get("trailingPegRatio")
        if (ev_to_ebitda == None or pe == None or peg == None):
            url =rf"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?limit=1&apikey={api_key}"
            ratios = requests.get(url).json()
            
            ev_to_ebitda = ratios[0].get("enterpriseValueOverEBITDA", 0)
            pe = ratios[0].get("peRatio", 0)
            peg = ratios[0].get("peRatio", 0) / financials[0].get("eps", 1)

    

        # Сбор данных для таблицы
        row = {
            
            "Ticker": ticker,
            #G
            "Market Price 5Y Change": stock.history(period="5y")["Close"].pct_change().sum(),

            "EPS 5Y Change": (info.get("forwardEps")/info.get("trailingEps", 0)),
            "YoY Growth": info.get("revenueGrowth"),

            "Multiplicative method": multiplicative_forecasting[ticker][0],
            "CAGR method": cagr_forecasting[ticker][0],

            #V
            
            "P/S": info.get("priceToSalesTrailing12Months"),
            "P/E": pe,
            "EV/EBITDA": ev_to_ebitda,
            "PEG": peg,
            # "ROI": info.get("returnOnInvestment"),
            "ROA": info.get("returnOnAssets"),
            "ROE": info.get("returnOnEquity"),
            
            #Q
            "DPR": info.get("payoutRatio", 0),
            "Dividend Yield": dividend_yield,
            "Cash Flow 5Y Change": cash_flow_5y_change,
            "EBITDA 5Y Change": ebitda_5y_change,
            "Sales 5Y Change": sales_5y_change,
            # "Net Income 5Y Change": net_income_5y_change,
            "Net Debt/EBITDA": info.get("totalDebt") / float(financials[0].get("ebitda", 1)),
            "FCF Yield": FCF_yield
            
        }
        data.append(row)

        sleep(3)

    return pd.DataFrame(data)



## ANALYSIS

In [28]:
def analysis(df):

    # 1. Нахождение максимальных значений для каждой метрики
    max_values = df.drop(columns=["Ticker"]).max()

    # 2. Нормализуем метрики по максимальному значению
    for col in df.columns[1:]:  # Все столбцы кроме "Ticker"
        if (col == "Net Debt/EBITDA"):
            df[col] = 1 - df[col] / max_values[col]
        else :
            df[col] = df[col] / max_values[col]

    # 3. Создание подгрупп для вычисления средних
    def calculate_group_averages(group_columns):
        return df[group_columns].mean(axis=1)

    # Перебираем ключи из gqv_for_aspects и вычисляем средние
    for group, aspects in gqv_for_aspects.items():
        group_columns = []
        
        # Собираем колонки для каждой подгруппы
        for aspect in aspects:
            group_columns.extend(aspects_metrics[aspect])  # Сразу добавляем метрики из списка

        # Вычисляем среднее для каждой группы
        df[f'{group}_avg'] = calculate_group_averages(group_columns)

    # 4. Вычисление среднего по меткам G, Q, V
    df['GQV_avg'] = df[['G_avg', 'V_avg', 'Q_avg']].mean(axis=1)


    return df


def unified_analysis(sectors_df):
    # 1. Объединение всех секторов в один DataFrame
    combined_df = pd.DataFrame()
    for sector, sector_data in sectors_df.items():
        if not sector_data.empty:
            sector_data["Sector"] = sector  # Добавляем столбец сектора
            combined_df = pd.concat([combined_df, sector_data], ignore_index=True)

    # 2. Нахождение глобальных максимальных значений
    max_values = combined_df.drop(columns=["Ticker", "Sector"]).max()

    # 3. Нормализация метрик
    for col in combined_df.columns:
        if col not in ["Ticker", "Sector"]:
            if col == "Net Debt/EBITDA":
                combined_df[col] = 1 - combined_df[col] / max_values[col]
            else:
                combined_df[col] = combined_df[col] / max_values[col]

    # 4. Создание подгрупп для вычисления средних
    def calculate_group_averages(group_columns):
        return combined_df[group_columns].mean(axis=1)

    for group, aspects in gqv_for_aspects.items():
        group_columns = []
        for aspect in aspects:
            group_columns.extend(aspects_metrics[aspect])  # Добавляем метрики из списка

        combined_df[f'{group}_avg'] = calculate_group_averages(group_columns)

    # 5. Вычисление среднего по G, Q, V
    combined_df['GQV_avg'] = combined_df[['G_avg', 'V_avg', 'Q_avg']].mean(axis=1)

    return combined_df


## VISUALIZATION

In [12]:
def plot_3d(df):
    # 5. Строим 3D график с Plotly
    fig = px.scatter_3d(df, x='G_avg', y='V_avg', z='Q_avg',
                        color='Ticker', hover_data=['Ticker'],
                        title='3D Analysis of Metrics')

    fig.update_layout(scene=dict(
                        xaxis_title='Growth (G)',
                        yaxis_title='Valuation (V)',
                        zaxis_title='Quality (Q)'
                        ))

    fig.show()

In [32]:


def plot_3d(df):
    # Добавляем точки MINIMUM и MAXIMUM
    # minimum_point = pd.DataFrame({'G_avg': [0], 'V_avg': [0], 'Q_avg': [0], 'Ticker': ['MINIMUM']})
    # maximum_point = pd.DataFrame({'G_avg': [1], 'V_avg': [1], 'Q_avg': [1], 'Ticker': ['MAXIMUM']})
    # df = pd.concat([df, minimum_point, maximum_point], ignore_index=True)

    # Строим 3D график с Plotly
    fig = px.scatter_3d(df, x='G_avg', y='V_avg', z='Q_avg',
                        color='Ticker', hover_data=['Ticker'],
                        title='3D Analysis of Metrics')

    fig.update_layout(scene=dict(
                        xaxis_title='Growth (G)',
                        yaxis_title='Valuation (V)',
                        zaxis_title='Quality (Q)',
                        xaxis=dict(range=[0, 1]),  # Ограничиваем оси от 0 до 1
                        yaxis=dict(range=[0, 1]),
                        zaxis=dict(range=[0, 1])
                        ))

    fig.show()


def plot_3d_analysis(df):
    fig = px.scatter_3d(
        df, 
        x='G_avg', 
        y='V_avg', 
        z='Q_avg',
        color='Sector',  # Цвет точек зависит от сектора
        hover_data=['Ticker', 'GQV_avg'],  # Информация, отображаемая при наведении
        title='3D Analysis of GQV Metrics Across Sectors'
    )

    fig.update_layout(
        scene=dict(
            xaxis_title='Growth (G)',
            yaxis_title='Valuation (V)',
            zaxis_title='Quality (Q)',
            # xaxis=dict(range=[0, 1]),  # Ограничиваем оси от 0 до 1
            # yaxis=dict(range=[0, 1]),
            # zaxis=dict(range=[0, 1])
        ),
        legend_title_text='Sector'
    )

    fig.show()






 # 

### ------------------------------------------------- 

# 

G
['Market Price 5Y Change', 'EPS 5Y Change', 'YoY Growth', 'Multiplicative method', 'CAGR method']
V
['P/S', 'P/E', 'EV/EBITDA', 'PEG']
Q
['DPR', 'Dividend Yield', 'Cash Flow 5Y Change', 'Net Income 5Y Change', 'EBITDA 5Y Change', 'FCF Yield', 'Sales 5Y Change', 'Net Debt/EBITDA']
ANALUYSIS COMPLETED


In [23]:
sectors_df = {
    "^SP500-35": pd.DataFrame(), 
    "^SP500-45": pd.DataFrame(), 
    "^SP500-30": pd.DataFrame(),
    "^SP500-25": pd.DataFrame(),
    "^SP500-55": pd.DataFrame(),
    "^SP500-60": pd.DataFrame(),
    "^GSPC": pd.DataFrame()
    }

initial_data_sectors_df = {
    "^SP500-35": pd.DataFrame(), 
    "^SP500-45": pd.DataFrame(), 
    "^SP500-30": pd.DataFrame(),
    "^SP500-25": pd.DataFrame(),
    "^SP500-55": pd.DataFrame(),
    "^SP500-60": pd.DataFrame(),
    "^GSPC": pd.DataFrame()
    }



In [15]:

sectors_tickers = {
    "^SP500-35": ['JNJ', 'NVO', 'PFE', 'SUPN', 'LLY'], 
    "^SP500-45": ['AAPL', 'MSFT', 'CFLT', 'ORCL', 'CRM', 'DDOG'], 
    "^SP500-30": ['PEP', 'PG', 'KO'],
    "^SP500-25": ['WMT', 'LOW', 'TGT', 'TSLA'],
    "^SP500-55": ['IBDRY', 'NEE', 'SO'],
    "^GSPC": ['MA', 'JPM', 'SEIC', 'PYPL', 'HD', 'WFC', 'BRK-B', 'UPST'],
    "^SP500-60": ['EQIX', 'AMT', 'SPG', 'AMT', 'ZG']
    }
sectors = [
    "^GSPC", 
    "^SP500-45",
    "^SP500-30",
    "^SP500-55",
    "^SP500-35",
    "^SP500-25",
    "^SP500-60"
    ]

# [
#     {'date': '2023-12-31', 
#      'symbol': 'UPST', 
#      'reportedCurrency': 'USD', 
#      'cik': '0001647639', 
#      'fillingDate': '2024-02-15', 
#      'acceptedDate': '2024-02-15 16:16:15',
#        'calendarYear': '2023', 
#        'period': 'FY', 
#        'revenue': 513562000, 'costOfRevenue': 24903000, 'grossProfit': 488659000, 'grossProfitRatio': 0.9515092628, 'researchAndDevelopmentExpenses': 280138000, 'generalAndAdministrativeExpenses': 212388000, 'sellingAndMarketingExpenses': 127143000, 'sellingGeneralAndAdministrativeExpenses': 339531000, 'otherExpenses': 150418000, 'operatingExpenses': 770087000, 'costAndExpenses': 770087000, 'interestIncome': 168996000, 'interestExpense': 34894000, 'depreciationAndAmortization': 24903000, 'ebitda': -215121999, 'ebitdaratio': -0.4188822362, 'operatingIncome': -256524999, 'operatingIncomeRatio': -0.4995015188, 'totalOtherIncomeExpensesNet': 16499999, 'incomeBeforeTax': -240025000, 'incomeBeforeTaxRatio': -0.4673729754, 'incomeTaxExpense': 107000, 'netIncome': -240132000, 'netIncomeRatio': -0.4675813242, 'eps': -2.87, 'epsdiluted': -2.87, 'weightedAverageShsOut': 83765896, 'weightedAverageShsOutDil': 83765896, 'link': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763924000009/0001647639-24-000009-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763924000009/upst-20231231.htm'}, {'date': '2022-12-31', 'symbol': 'UPST', 'reportedCurrency': 'USD', 'cik': '0001647639', 'fillingDate': '2023-02-16', 'acceptedDate': '2023-02-16 15:49:35', 'calendarYear': '2022', 'period': 'FY', 'revenue': 853287000, 'costOfRevenue': 187994000, 'grossProfit': 665293000, 'grossProfitRatio': 0.7796825687, 'researchAndDevelopmentExpenses': 237247000, 'generalAndAdministrativeExpenses': 185290000, 'sellingAndMarketingExpenses': 345776000, 'sellingGeneralAndAdministrativeExpenses': 531066000, 'otherExpenses': 9473000, 'operatingExpenses': 772997000, 'costAndExpenses': 960991000, 'interestIncome': 105580000, 'interestExpense': 10843000, 'depreciationAndAmortization': 13513000, 'ebitda': -89507000, 'ebitdaratio': -0.1048967112, 'operatingIncome': -103020000, 'operatingIncomeRatio': -0.1207331179, 'totalOtherIncomeExpensesNet': 4789000, 'incomeBeforeTax': -109074000, 'incomeBeforeTaxRatio': -0.1278280344, 'incomeTaxExpense': 409000, 'netIncome': -108665000, 'netIncomeRatio': -0.1273487115, 'eps': -1.31, 'epsdiluted': -1.31, 'weightedAverageShsOut': 82771268, 'weightedAverageShsOutDil': 82771268, 'link': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763923000021/0001647639-23-000021-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763923000021/upst-20221231.htm'}, {'date': '2021-12-31', 'symbol': 'UPST', 'reportedCurrency': 'USD', 'cik': '0001647639', 'fillingDate': '2022-02-18', 'acceptedDate': '2022-02-17 19:36:15', 'calendarYear': '2021', 'period': 'FY', 'revenue': 851863000, 'costOfRevenue': 117579000, 'grossProfit': 734284000, 'grossProfitRatio': 0.8619742846, 'researchAndDevelopmentExpenses': 133999000, 'generalAndAdministrativeExpenses': 121440000, 'sellingAndMarketingExpenses': 333453000, 'sellingGeneralAndAdministrativeExpenses': 454893000, 'otherExpenses': -5174000, 'operatingExpenses': 597305000, 'costAndExpenses': 714884000, 'interestIncome': 47314000, 'interestExpense': 3274000, 'depreciationAndAmortization': 7541000, 'ebitda': 140881000, 'ebitdaratio': 0.179527694, 'operatingIncome': 145392000, 'operatingIncomeRatio': 0.1706753316, 'totalOtherIncomeExpensesNet': -7150000, 'incomeBeforeTax': 133731000, 'incomeBeforeTaxRatio': 0.1569865107, 'incomeTaxExpense': 1712000, 'netIncome': 135443000, 'netIncomeRatio': 0.1589962236, 'eps': 1.73, 'epsdiluted': 1.43, 'weightedAverageShsOut': 78106359, 'weightedAverageShsOutDil': 94772641, 'link': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763922000009/0001647639-22-000009-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763922000009/upst-20211231.htm'}, {'date': '2020-12-31', 'symbol': 'UPST', 'reportedCurrency': 'USD', 'cik': '0001647639', 'fillingDate': '2021-03-18', 'acceptedDate': '2021-03-18 16:42:45', 'calendarYear': '2020', 'period': 'FY', 'revenue': 241442000, 'costOfRevenue': 37581000, 'grossProfit': 203861000, 'grossProfitRatio': 0.8443477108, 'researchAndDevelopmentExpenses': 38802000, 'generalAndAdministrativeExpenses': 45609000, 'sellingAndMarketingExpenses': 99659000, 'sellingGeneralAndAdministrativeExpenses': 145268000, 'otherExpenses': 5549000, 'operatingExpenses': 190134000, 'costAndExpenses': 227715000, 'interestIncome': 4816000, 'interestExpense': 8026000, 'depreciationAndAmortization': 2278000, 'ebitda': 11765000, 'ebitdaratio': 0.0914049751, 'operatingIncome': 19791000, 'operatingIncomeRatio': 0.0819699969, 'totalOtherIncomeExpensesNet': -5815000, 'incomeBeforeTax': 5950000, 'incomeBeforeTaxRatio': 0.0246435997, 'incomeTaxExpense': 371000, 'netIncome': 5983000, 'netIncomeRatio': 0.0247802785, 'eps': 0.0816, 'epsdiluted': 0.0816, 'weightedAverageShsOut': 73314026, 'weightedAverageShsOutDil': 73314026, 'link': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763921000004/0001647639-21-000004-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1647639/000164763921000004/upstrtholdingsinc202010-k.htm'}, 
       
       
#     {'date': '2019-12-31', 
#      'symbol': 'UPST', 
#      'reportedCurrency': 'USD', 
#      'cik': '0001647639', 
#      'fillingDate': '2019-12-31', 
#      'acceptedDate': '2019-12-31 00:00:00', 
#      'calendarYear': '2019', 'period': 'FY', 
#      'revenue': 190674000, 'costOfRevenue': 24947000, 'grossProfit': 165727000, 'grossProfitRatio': 0.8691641231, 
#      'researchAndDevelopmentExpenses': 18777000, 'generalAndAdministrativeExpenses': 31865000, 
#      'sellingAndMarketingExpenses': 93175000, 'sellingGeneralAndAdministrativeExpenses': 125040000, 
#      'otherExpenses': 1036000, 'operatingExpenses': 145224000, 'costAndExpenses': 170171000, s
#      'interestIncome': 63313000, 'interestExpense': 26485000, 'depreciationAndAmortization': 774000, 
#      'ebitda': 0, 'ebitdaratio': 0.1189674523, 'operatingIncome': 21910000, 'operatingIncomeRatio': 0.1149081679, 'totalOtherIncomeExpensesNet': -4946000, 'incomeBeforeTax': -4946000, 'incomeBeforeTaxRatio': -0.0259395618, 'incomeTaxExpense': 74000, 'netIncome': -466000, 'netIncomeRatio': -0.0024439619, 'eps': -0.0064, 'epsdiluted': -0.0064, 'weightedAverageShsOut': 73314026, 'weightedAverageShsOutDil': 73314026, 'link': None, 'finalLink': None}]


# [
#     {'date': '2023-12-31', 'symbol': 'WFC', 'reportedCurrency': 'USD', 'cik': '0000072971', 'fillingDate': '2024-02-20', 'acceptedDate': '2024-02-20 17:06:58', 'calendarYear': '2023', 'period': 'FY', 'revenue': 77198000000, 'costOfRevenue': -5399000000, 'grossProfit': 82597000000, 'grossProfitRatio': 1.069937045, 'researchAndDevelopmentExpenses': 0, 'generalAndAdministrativeExpenses': 35829000000, 'sellingAndMarketingExpenses': 812000000, 'sellingGeneralAndAdministrativeExpenses': 48530000000, 'otherExpenses': 0, 'operatingExpenses': 48530000000, 'costAndExpenses': 48530000000, 'interestIncome': 85118000000, 'interestExpense': 32743000000, 'depreciationAndAmortization': 6271000000, 'ebitda': 27907000000, 'ebitdaratio': 0.3614990026, 'operatingIncome': 28668000000, 'operatingIncomeRatio': 0.3713567709, 'totalOtherIncomeExpensesNet': -7032000000, 'incomeBeforeTax': 21636000000, 'incomeBeforeTaxRatio': 0.2802663281, 'incomeTaxExpense': 2607000000, 'netIncome': 19142000000, 'netIncomeRatio': 0.2479597917, 'eps': 4.88, 'epsdiluted': 4.83, 'weightedAverageShsOut': 3688300000, 'weightedAverageShsOutDil': 3720400000, 'link': 'https://www.sec.gov/Archives/edgar/data/72971/000007297124000064/0000072971-24-000064-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/72971/000007297124000064/wfc-20231231.htm'}, {'date': '2022-12-31', 'symbol': 'WFC', 'reportedCurrency': 'USD', 'cik': '0000072971', 'fillingDate': '2023-02-21', 'acceptedDate': '2023-02-21 17:13:29', 'calendarYear': '2022', 'period': 'FY', 'revenue': 73785000000, 'costOfRevenue': 0, 'grossProfit': 73785000000, 'grossProfitRatio': 1, 'researchAndDevelopmentExpenses': 0, 'generalAndAdministrativeExpenses': 5188000000, 'sellingAndMarketingExpenses': 505000000, 'sellingGeneralAndAdministrativeExpenses': 46289000000, 'otherExpenses': 10989000000, 'operatingExpenses': 57278000000, 'costAndExpenses': 57278000000, 'interestIncome': 54024000000, 'interestExpense': 9074000000, 'depreciationAndAmortization': 7309000000, 'ebitda': 22278000000, 'ebitdaratio': 0.4057735312, 'operatingIncome': 14970000000, 'operatingIncomeRatio': 0.2028867656, 'totalOtherIncomeExpensesNet': -6832000000, 'incomeBeforeTax': 14969000000, 'incomeBeforeTaxRatio': 0.2028732127, 'incomeTaxExpense': 2087000000, 'netIncome': 13677000000, 'netIncomeRatio': 0.1853628786, 'eps': 3.17, 'epsdiluted': 3.14, 'weightedAverageShsOut': 3805200000, 'weightedAverageShsOutDil': 3837000000, 'link': 'https://www.sec.gov/Archives/edgar/data/72971/000007297123000071/0000072971-23-000071-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/72971/000007297123000071/wfc-20221231.htm'}, {'date': '2021-12-31', 'symbol': 'WFC', 'reportedCurrency': 'USD', 'cik': '0000072971', 'fillingDate': '2022-02-22', 'acceptedDate': '2022-02-22 17:01:56', 'calendarYear': '2021', 'period': 'FY', 'revenue': 78492000000, 'costOfRevenue': 0, 'grossProfit': 78492000000, 'grossProfitRatio': 1, 'researchAndDevelopmentExpenses': 0, 'generalAndAdministrativeExpenses': 5723000000, 'sellingAndMarketingExpenses': 600000000, 'sellingGeneralAndAdministrativeExpenses': 48059000000, 'otherExpenses': 5731000000, 'operatingExpenses': 53790000000, 'costAndExpenses': 53790000000, 'interestIncome': 39694000000, 'interestExpense': 3915000000, 'depreciationAndAmortization': 8494000000, 'ebitda': 37310000000, 'ebitdaratio': 0.7351322428, 'operatingIncome': 28851000000, 'operatingIncomeRatio': 0.3675661214, 'totalOtherIncomeExpensesNet': -6446000000, 'incomeBeforeTax': 28816000000, 'incomeBeforeTaxRatio': 0.3671202161, 'incomeTaxExpense': 5578000000, 'netIncome': 22109000000, 'netIncomeRatio': 0.2816720175, 'eps': 4.99, 'epsdiluted': 4.95, 'weightedAverageShsOut': 4061900000, 'weightedAverageShsOutDil': 4096200000, 'link': 'https://www.sec.gov/Archives/edgar/data/72971/000007297122000096/0000072971-22-000096-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/72971/000007297122000096/wfc-20211231.htm'}, {'date': '2020-12-31', 'symbol': 'WFC', 'reportedCurrency': 'USD', 'cik': '0000072971', 'fillingDate': '2021-02-23', 'acceptedDate': '2021-02-23 16:58:04', 'calendarYear': '2020', 'period': 'FY', 'revenue': 72340000000, 'costOfRevenue': 0, 'grossProfit': 72340000000, 'grossProfitRatio': 1, 'researchAndDevelopmentExpenses': 0, 'generalAndAdministrativeExpenses': 6706000000, 'sellingAndMarketingExpenses': 600000000, 'sellingGeneralAndAdministrativeExpenses': 48479000000, 'otherExpenses': 9188000000, 'operatingExpenses': 57667000000, 'costAndExpenses': 57667000000, 'interestIncome': 47798000000, 'interestExpense': 7963000000, 'depreciationAndAmortization': 8974000000, 'ebitda': 0, 'ebitdaratio': 0.1117224219, 'operatingIncome': 4041000000, 'operatingIncomeRatio': 0.0558612109, 'totalOtherIncomeExpensesNet': -9188000000, 'incomeBeforeTax': 2505000000, 'incomeBeforeTaxRatio': 0.0346281449, 'incomeTaxExpense': 1157000000, 'netIncome': 1786000000, 'netIncomeRatio': 0.0246889688, 'eps': 0.43, 'epsdiluted': 0.43, 'weightedAverageShsOut': 4118000000, 'weightedAverageShsOutDil': 4134200000, 'link': 'https://www.sec.gov/Archives/edgar/data/72971/000007297121000197/0000072971-21-000197-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/72971/000007297121000197/wfc-20201231.htm'}, {'date': '2019-12-31', 'symbol': 'WFC', 'reportedCurrency': 'USD', 'cik': '0000072971', 'fillingDate': '2020-02-27', 'acceptedDate': '2020-02-26 21:51:08', 'calendarYear': '2019', 'period': 'FY', 'revenue': 85063000000, 'costOfRevenue': 0, 'grossProfit': 85063000000, 'grossProfitRatio': 1, 'researchAndDevelopmentExpenses': 0, 'generalAndAdministrativeExpenses': 6745000000, 'sellingAndMarketingExpenses': 1076000000, 'sellingGeneralAndAdministrativeExpenses': 49170000000, 'otherExpenses': 9090000000, 'operatingExpenses': 58260000000, 'costAndExpenses': 58260000000, 'interestIncome': 66083000000, 'interestExpense': 18852000000, 'depreciationAndAmortization': 7923000000, 'ebitda': 0, 'ebitdaratio': 0.6105357206, 'operatingIncome': 25967000000, 'operatingIncomeRatio': 0.3052678603, 'totalOtherIncomeExpensesNet': -9090000000, 'incomeBeforeTax': 25967000000, 'incomeBeforeTaxRatio': 0.3052678603, 'incomeTaxExpense': 5761000000, 'netIncome': 19715000000, 'netIncomeRatio': 0.2317693944, 'eps': 4.08, 'epsdiluted': 4.05, 'weightedAverageShsOut': 4393100000, 'weightedAverageShsOutDil': 4425400000, 'link': 'https://www.sec.gov/Archives/edgar/data/72971/000007297120000217/0000072971-20-000217-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/72971/000007297120000217/wfc-12312019x10k.htm'}]


### FINANCE

In [24]:
for sector in sectors:

    if (len(initial_data_sectors_df[sector].index) > len(pd.DataFrame().index)):
        df = initial_data_sectors_df[sector]
    else:
        df = get_data(sector_ticker=sector, tickers=sectors_tickers[sector])
        print("GET DATA COMPLETED")
        initial_data_sectors_df[sector] = df

    
    df = analysis(df = df)
    print("ANALUYSIS COMPLETED")
    sectors_df[sector] = df
    


MA
[{'date': '2023-12-31', 'symbol': 'MA', 'reportedCurrency': 'USD', 'cik': '0001141391', 'fillingDate': '2024-02-13', 'acceptedDate': '2024-02-13 12:47:32', 'calendarYear': '2023', 'period': 'FY', 'revenue': 25098000000, 'costOfRevenue': 0, 'grossProfit': 25098000000, 'grossProfitRatio': 1, 'researchAndDevelopmentExpenses': 0, 'generalAndAdministrativeExpenses': 8927000000, 'sellingAndMarketingExpenses': 825000000, 'sellingGeneralAndAdministrativeExpenses': 9752000000, 'otherExpenses': 1338000000, 'operatingExpenses': 11090000000, 'costAndExpenses': 11090000000, 'interestIncome': 0, 'interestExpense': 575000000, 'depreciationAndAmortization': 799000000, 'ebitda': 15148000000, 'ebitdaratio': 0.6035540681, 'operatingIncome': 14008000000, 'operatingIncomeRatio': 0.5581321221, 'totalOtherIncomeExpensesNet': -369000000, 'incomeBeforeTax': 13639000000, 'incomeBeforeTaxRatio': 0.5434297554, 'incomeTaxExpense': 2444000000, 'netIncome': 11195000000, 'netIncomeRatio': 0.4460514782, 'eps': 11.8

In [4]:
sectors = [
    "^GSPC", 
    "^SP500-45",
    "^SP500-30",
    "^SP500-55",
    "^SP500-35",
    "^SP500-25",
    "^SP500-60"
    ]

plot_3d(df = sectors_df["^SP500-60"])
sectors_df["^SP500-60"]

NameError: name 'plot_3d' is not defined

In [3]:
sectors_df["^GSPC"] = df

NameError: name 'df' is not defined

## Analyse total DFs

In [45]:
def rank_and_categorize_stocks(sectors_df, top_n=10):
    results = {}

    for sector, sector_data in sectors_df.items():
        if sector_data.empty:
            continue

        # Создаем новый столбец для итогового балла
        sector_data['score'] = 0

        # Сортируем по каждому из параметров и GQV_avg
        rankings = {}
        for col in ['G_avg', 'V_avg', 'Q_avg', 'GQV_avg']:
            rankings[col] = sector_data.sort_values(by=col, ascending=False).head(top_n)

            # Применяем баллы по позиции для топ-3
            for i, row in rankings[col].iterrows():
                if i == 0:
                    sector_data.loc[sector_data['Ticker'] == row['Ticker'], 'score'] += 5
                elif i == 1:
                    sector_data.loc[sector_data['Ticker'] == row['Ticker'], 'score'] += 4
                elif i == 2:
                    sector_data.loc[sector_data['Ticker'] == row['Ticker'], 'score'] += 3

        # Обработка нейтральных акций
        neutral_tickers = pd.concat(rankings.values())['Ticker'].unique()
        neutral_stocks = sector_data[~sector_data['Ticker'].isin(neutral_tickers)]
        sector_data.loc[neutral_stocks.index, 'score'] += 1  # Нейтральные акции

        # Обработка аутсайдеров (отрицательный GQV_avg)
        underperformers = sector_data[sector_data['GQV_avg'] < 0]
        sector_data.loc[underperformers.index, 'score'] = -2  # Аутсайдеры

        # Сохраняем результаты
        results[sector] = {
            "rankings": rankings,
            "underperformers": underperformers,
            "neutral_stocks": neutral_stocks,
            "sector_data": sector_data  # Добавляем обновленные данные с баллами
        }

    return results


def print_sector_rankings(results):
    for sector, data in results.items():
        print(f"=== Sector: {sectors_names[sector]} ===")
        
        # Вывод топов с баллами
        print("\nTop stocks by metrics with scores:")
        for metric, ranking in data["rankings"].items():
            print(f"  - {metric} (Top {len(ranking)}):")
            for idx, row in ranking.iterrows():
                print(f"    {row['Ticker']} - {metric}: {row[metric]:.2f}")

        # Вывод аутсайдеров
        if not data["underperformers"].empty:
            print("\nUnderperformers (Negative GQV_avg):")
            for idx, row in data["underperformers"].iterrows():
                print(f"  {row['Ticker']} - GQV_avg: {row['GQV_avg']:.2f}")

        # Вывод нейтральных акций
        if not data["neutral_stocks"].empty:
            print("\nNeutral stocks (Not in Top and not underperformers):")
            for idx, row in data["neutral_stocks"].iterrows():
                print(f"  {row['Ticker']} - GQV_avg: {row['GQV_avg']:.2f}")

        # Вывод всех акций по баллам (для наглядности)
        # print("\nAll stocks sorted by score:")
        # sorted_by_score = data["sector_data"].sort_values(by="score", ascending=False)
        # for idx, row in sorted_by_score.iterrows():
        #     print(f"  {row['Ticker']} - Score: {row['score']}")

        print("\n")

def print_all_stocks_by_gqv(results):
    for sector, data in results.items():
        print(f"=== Sector: {sectors_names[sector]} ===")
        
        # Сортируем акции по убыванию GQV_avg
        sorted_by_gqv = data["sector_data"].sort_values(by="GQV_avg", ascending=False)
        
        # Выводим акции с их G_avg, V_avg, Q_avg и GQV_avg
        print("\nStocks sorted by GQV_avg:")
        for idx, row in sorted_by_gqv.iterrows():
            print(f"  {row['Ticker']} - GQV_avg: {row['GQV_avg']:.2f} | "
                  f"G_avg: {row['G_avg']:.2f} | V_avg: {row['V_avg']:.2f} | Q_avg: {row['Q_avg']:.2f}")
        
        print("\n")



# Использование:
sector_rankings = rank_and_categorize_stocks(sectors_df, top_n=3)
# print_sector_rankings(sector_rankings)

print_all_stocks_by_gqv(sector_rankings)



=== Sector: health ===

Stocks sorted by GQV_avg:
  LLY - GQV_avg: 0.51 | G_avg: 0.79 | V_avg: 0.52 | Q_avg: 0.22
  NVO - GQV_avg: 0.49 | G_avg: 0.64 | V_avg: 0.05 | Q_avg: 0.76
  SUPN - GQV_avg: 0.42 | G_avg: 0.45 | V_avg: 0.64 | Q_avg: 0.18
  PFE - GQV_avg: 0.29 | G_avg: 0.53 | V_avg: 0.10 | Q_avg: 0.23
  JNJ - GQV_avg: 0.21 | G_avg: 0.23 | V_avg: 0.14 | Q_avg: 0.26


=== Sector: bigtech ===

Stocks sorted by GQV_avg:
  CFLT - GQV_avg: 0.39 | G_avg: 0.41 | V_avg: 0.36 | Q_avg: 0.41
  MSFT - GQV_avg: 0.38 | G_avg: 0.36 | V_avg: 0.25 | Q_avg: 0.52
  CRM - GQV_avg: 0.37 | G_avg: 0.35 | V_avg: 0.23 | Q_avg: 0.53
  ORCL - GQV_avg: 0.34 | G_avg: 0.34 | V_avg: 0.22 | Q_avg: 0.48
  AAPL - GQV_avg: 0.32 | G_avg: 0.31 | V_avg: 0.22 | Q_avg: 0.44
  DDOG - GQV_avg: 0.11 | G_avg: 1.00 | V_avg: 0.77 | Q_avg: -1.43


=== Sector: staples ===

Stocks sorted by GQV_avg:
  PG - GQV_avg: 0.84 | G_avg: 0.90 | V_avg: 0.94 | Q_avg: 0.68
  KO - GQV_avg: 0.83 | G_avg: 0.91 | V_avg: 0.89 | Q_avg: 0.68
  PEP -

In [33]:
unified_df = unified_analysis(sectors_df=initial_data_sectors_df)

plot_3d_analysis(unified_df)

In [ ]:

# Использование функции
sector_rankings = rank_and_categorize_stocks(df, sectors_df)
